# Install/import relevant libraries

In [18]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle
from scipy import spatial
import os
print(os.getcwd())

/content


In [19]:
!pip install sentence-transformers

# Get embeddings for distilbert

In [20]:
# qna_df = pd.read_csv("../../0.Datasets/QnA.csv")
qna_df = pd.read_csv('./QnA.csv')

In [21]:
qna_df.head()

,Unnamed: 0,qid,docid,question,answer
0,0,0,18850,What is considered a business expense on a bus...,The IRS Guidance pertaining to the subject. I...
1,1,1,14255,Claiming business expenses for a business with...,Yes you can claim your business deductions if ...
2,2,2,308938,Transferring money from One business checking ...,You should have separate files for each of the...
3,3,3,296717,Having a separate bank account for business/in...,Having a separate checking account for the bus...
4,4,3,100764,Having a separate bank account for business/in...,"You don't specify which country you are in, so..."


In [22]:
qna_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17072 entries, 0 to 17071
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  17072 non-null  int64 
 1   qid         17072 non-null  int64 
 2   docid       17072 non-null  int64 
 3   question    17072 non-null  object
 4   answer      17072 non-null  object
dtypes: int64(3), object(2)
memory usage: 667.0+ KB


In [24]:
# model = SentenceTransformer("sentence-transformers/msmarco-distilbert-dot-v5")
model = SentenceTransformer("sentence-transformers/msmarco-bert-base-dot-v5")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/636 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/461 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [25]:
# split answers to chunks for model to encode
answers_list = qna_df["answer"].tolist()
chunk_size = 10
chunks = [answers_list[i:(i+chunk_size)] for i in range(0, len(answers_list), chunk_size)]

In [26]:
answers_list = qna_df["answer"].tolist()

In [27]:
dimension_size = 768 # embedding dimensions from msmarco distilbert documentation
sentence_embeddings = np.empty([0,dimension_size], dtype=np.float32)
for i,chunk in enumerate(chunks):
    if i% chunk_size == 0:
        print(i* chunk_size)
    # stack it to sentence_embeddings numpy array
    sentence_embeddings = np.vstack((sentence_embeddings, model.encode(chunk)))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000


In [28]:
len(sentence_embeddings)

17072

In [29]:
#save model embeddings
with open("./embeddings_bertbase.pkl", 'wb') as f:
    pickle.dump(sentence_embeddings, f, pickle.HIGHEST_PROTOCOL)

# Show top 50 results for a query

In [15]:
# Read embeddings
sentence_embeddings = pickle.load(open("./embeddings.pkl", 'rb'))

In [16]:
test_query = "Should i use a robo advisor or hire a financial advisor?"

In [30]:
%%time
question_embedding = model.encode(test_query)
answer_similiarity = {}
for i,embed in enumerate(sentence_embeddings):
    answer_similiarity[i]= np.dot(question_embedding, embed)
answer_similiarity = {k: v for k, v in sorted(answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
top_20_index = list(answer_similiarity)[:50]
for i,index in enumerate(top_20_index):
    print(f"========== Rank {i+1} ==========")
    print(answers_list[index])
    print()

========== Rank 1 ==========
If your financial needs aren't complex, and mostly limited to portfolio management, consider looking into the newish thing called robo-advisers (proper term is "Automated investing services").  The difference is that robo-advisers use software to manage portfolios on a large scale, generating big economy of scale and therefore offering a much cheaper services than personal advisor would - and unless your financial needs are extremely complex, the state of the art of scaled up portfolio management is at the point that a human advisor really doesn't give you any value-add (and - as other answers noted - human advisor can easily bring in downsides such as conflict of interest and lack of fiduciary responsibility).  disclaimer: I indirectly derive my living from a company which derives a very small part of their income from a robo-adviser, therefore there's a possible small conflict of interest in my answer

========== Rank 2 ==========
If you are looking for a

In [39]:
# Save answer
df = pd.DataFrame(columns=['Rank', 'Index', 'Answer'])
for i,index in enumerate(top_20_index):
    print(f"========== Rank {i+1} ==========")
    print(answers_list[index])
    print()
    df = df.append({'Rank': i+1, 'Index': index, 'Answer': answers_list[index]}, ignore_index=True)

df.to_csv('bertbase_answers.csv', index=False)
df.head()

========== Rank 1 ==========
If your financial needs aren't complex, and mostly limited to portfolio management, consider looking into the newish thing called robo-advisers (proper term is "Automated investing services").  The difference is that robo-advisers use software to manage portfolios on a large scale, generating big economy of scale and therefore offering a much cheaper services than personal advisor would - and unless your financial needs are extremely complex, the state of the art of scaled up portfolio management is at the point that a human advisor really doesn't give you any value-add (and - as other answers noted - human advisor can easily bring in downsides such as conflict of interest and lack of fiduciary responsibility).  disclaimer: I indirectly derive my living from a company which derives a very small part of their income from a robo-adviser, therefore there's a possible small conflict of interest in my answer

========== Rank 2 ==========
If you are looking for a

,Rank,Index,Answer
0,1,8079,"If your financial needs aren't complex, and mo..."
1,2,601,If you are looking for an advisor to just buil...
2,3,8080,Most individuals do not need a personal financ...
3,4,10050,"Yes, there is a profession that does exactly w..."
4,5,10802,"The way this works, as I understand it, is tha..."
